Вычислите пять товаров с наибольшими продажами（По количестве）. Выберите товар, занимающий первое место.

In [15]:
import pandas as pd

# Загрузка данных
df = pd.read_csv('transactions_diy.csv')

# Группировка по товару, суммируем quantity, сортируем по убыванию
top_5_by_quantity = df.groupby('item', as_index=False)['quantity'].sum().sort_values(by='quantity', ascending=False).head(5)

# Первое место
top_1_item = top_5_by_quantity.iloc[0]['item']

# Вывод результата
print(f"Товар с наибольшим количеством продаж: {top_1_item}")


Товар с наибольшим количеством продаж: sku27292


Подсчитайте количество уникальных товаров (уникальных значений в столбце item) в файле с транзакциями.

In [20]:
import pandas as pd

# Загружаем данные
df = pd.read_csv('transactions_diy.csv')# Подсчёт уникальных значений в столбце 'item'
unique_items_count = df['item'].nunique()

# Вывод результата
print(f"Количество уникальных товаров (уникальных значений в столбце 'item'): {unique_items_count}")


Количество уникальных товаров (уникальных значений в столбце 'item'): 30566


Проанализируйте данные о продажах за последние 3 месяца, определите товар с наибольшим объемом продаж （По сумме）.

In [2]:
import pandas as pd

# Загрузка данных
df = pd.read_csv('transactions_diy.csv')
df['tr_date'] = pd.to_datetime(df['tr_date'], format='%d.%m.%Y')

# Фильтр: последние 3 месяца
last_date = df['tr_date'].max()
start_date = last_date - pd.DateOffset(months=3)
df_3m = df[df['tr_date'] >= start_date]

# Группировка и сумма по 'amount'
group_col = 'sku' if 'sku' in df.columns else 'item'
sales = df_3m.groupby(group_col)['amount'].sum()

# Лидер по сумме
leader_sku = sales.idxmax()
leader_amount = sales.max()

# Вывод ТОЛЬКО нужного результата
print(f"Лидер по сумме продаж:\n   Товар: {leader_sku}")


Лидер по сумме продаж:
   Товар: sku26161


Выберите все данные за май 2018 года, затем выполните подсчет уникальных значений по столбцу счетов.


In [11]:
import pandas as pd

# Загрузка и преобразование даты
df = pd.read_csv('transactions_diy.csv')
df['tr_date'] = pd.to_datetime(df['tr_date'], format='%d.%m.%Y')

# Фильтр: май 2018
df_may_2018 = df[(df['tr_date'] >= '2018-05-01') & (df['tr_date'] <= '2018-05-31')]

# Проверка bcode
if 'bcode' not in df_may_2018.columns:
    raise KeyError("❌ В данных нет колонки 'bcode'")

# Очистка bcode
df_may_2018['bcode'] = df_may_2018['bcode'].astype(str).str.strip()
valid = df_may_2018[
    (df_may_2018['bcode'] != 'nan') &
    (df_may_2018['bcode'] != '') &
    df_may_2018['bcode'].notna()
]

# Прямой подсчёт уникальных
unique_count = valid['bcode'].nunique()

print(f"🎯 Уникальных магазинов (bcode) в мае 2018: {unique_count}")


🎯 Уникальных магазинов (bcode) в мае 2018: 9315


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11900\390359210.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_may_2018['bcode'] = df_may_2018['bcode'].astype(str).str.strip()


Дополни XYZ-анализ товаров (по количеству): Используй 
для расчета коэффициент вариации = std / mean * 100. На 
основе коэффициента вариации распредели товары: - 
Группа X: CV < 10% - Группа Y: 10% <= CV < 25% - 
Группа Z: CV > 25%. Выведи количество товаров в каждой 
группе и итоговый датафрейм.Сколько товаров имеет группу "Z" по итогам XYZ-анализа на всех имеющихся данных?

In [4]:
import pandas as pd
import numpy as np

# --- 1. Загрузка данных (если ещё не загружены) ---
df = pd.read_csv('transactions_diy.csv')
df['tr_date'] = pd.to_datetime(df['tr_date'], format='%d.%m.%Y')

# --- 2. Проверим, есть ли нужные колонки ---
if 'item' not in df.columns:
    raise KeyError("❌ Нет колонки 'item'")
if 'quantity' not in df.columns:
    raise KeyError("❌ Нет колонки 'quantity'")

# --- 3. Группировка по товару (item) ---
xyz_data = df.groupby('item')['quantity'].agg(['mean', 'std']).round(3)

# Переименуем колонки
xyz_data.columns = ['mean_quantity', 'std_quantity']

# --- 4. Удаляем строки, где mean == 0 ---
xyz_data = xyz_data[xyz_data['mean_quantity'] > 0]

# --- 5. Расчёт коэффициента вариации (CV) ---
xyz_data['cv_percent'] = (xyz_data['std_quantity'] / xyz_data['mean_quantity']) * 100

# Заменяем NaN (если std = 0) и бесконечности...
xyz_data['cv_percent'] = xyz_data['cv_percent'].replace([np.inf, -np.inf], 0).fillna(0)

# --- 6. Присвоение группы XYZ ---
def assign_xyz(cv):
    if cv < 10:
        return 'X'
    elif 10 <= cv < 25:
        return 'Y'
    else:
        return 'Z'

xyz_data['Группа_XYZ'] = xyz_data['cv_percent'].apply(assign_xyz)

# --- 7. Вывод: количество товаров в каждой группе ---
group_counts = xyz_data['Группа_XYZ'].value_counts().reindex(['X', 'Y', 'Z'], fill_value=0)

print("📊 Количество товаров по группам XYZ:")
print("=" * 40)
print(group_counts)

# --- 8. Сколько товаров в группе Z? ---
z_count = group_counts['Z']
print(f"\n🎯 Количество товаров в группе Z: {z_count}")




📊 Количество товаров по группам XYZ:
Группа_XYZ
X    11467
Y     1537
Z    17562
Name: count, dtype: int64

🎯 Количество товаров в группе Z: 17562


Получите ABC-XYZ-матрицу на данных за последние 6 месяцев. Сколько товаров попало в сегмент CZ?

In [3]:
import pandas as pd
import numpy as np

# --- 1. Загрузка данных (если ещё не загружены) ---
# df = pd.read_csv('transactions_diy.csv')
# df['tr_date'] = pd.to_datetime(df['tr_date'])

# --- 2. Фильтрация: последние 6 месяцев ---
# Определяем дату начала (6 месяцев назад от последней даты)
last_date = df['tr_date'].max()
start_date = last_date - pd.DateOffset(months=6)
df_last_6m = df[df['tr_date'] >= start_date].copy()

print(f"📅 Анализ за период: с {start_date.date()} по {last_date.date()}")
print(f"📊 Количество транзакций: {len(df_last_6m)}")

# --- 3. Проверка данных ---
if 'item' not in df_last_6m.columns:
    raise KeyError("❌ Нет колонки 'item'")
if 'amount' not in df_last_6m.columns:
    raise KeyError("❌ Нет колонки 'amount'")
if 'quantity' not in df_last_6m.columns:
    raise KeyError("❌ Нет колонки 'quantity'")

# --- 4. ABC-анализ по сумме продаж (amount) ---
abc = df_last_6m.groupby('item')['amount'].sum().sort_values(ascending=False)
total_sales = abc.sum()
abc_cum_percent = (abc.cumsum() / total_sales) * 100

def assign_abc(cum_percent):
    if cum_percent <= 80:
        return 'A'
    elif cum_percent <= 95:
        return 'B'
    else:
        return 'C'

abc_groups = pd.Series(abc_cum_percent).apply(assign_abc)
abc_df = pd.DataFrame({'Группа_ABC': abc_groups})

# --- 5. XYZ-анализ по стабильности количества (quantity) ---
xyz_stats = df_last_6m.groupby('item')['quantity'].agg(['mean', 'std'])
xyz_stats = xyz_stats[xyz_stats['mean'] > 0]
xyz_stats['cv'] = (xyz_stats['std'] / xyz_stats['mean']) * 100
xyz_stats['cv'] = xyz_stats['cv'].replace([np.inf, -np.inf], 0).fillna(0)

def assign_xyz(cv):
    if cv < 10:
        return 'X'
    elif 10 <= cv < 25:
        return 'Y'
    else:
        return 'Z'

xyz_groups = xyz_stats['cv'].apply(assign_xyz)
xyz_df = pd.DataFrame({'Группа_XYZ': xyz_groups})

# --- 6. Объединение ABC и XYZ ---
abc_xyz = pd.concat([abc_df, xyz_df], axis=1, join='inner')

# --- 7. Создание матрицы ABC × XYZ ---
matrix = pd.crosstab(abc_xyz['Группа_ABC'], abc_xyz['Группа_XYZ'])

print("\n📋 Матрица ABC × XYZ (количество товаров):")
print(matrix)

# --- 8. Сколько товаров в сегменте CZ? ---
cz_count = matrix.loc['C', 'Z'] if ('C' in matrix.index and 'Z' in matrix.columns) else 0
print(f"\n🎯 Количество товаров в сегменте CZ: {cz_count}")

# --- 9. Итоговый датафрейм (опционально) ---
print(f"\n📋 Примеры товаров из сегмента CZ:")
cz_items = abc_xyz[(abc_xyz['Группа_ABC'] == 'C') & (abc_xyz['Группа_XYZ'] == 'Z')]
print(cz_items.index.tolist()[:10], "..." if len(cz_items) > 10 else "")


📅 Анализ за период: с 2019-04-30 по 2019-10-31
📊 Количество транзакций: 350864

📋 Матрица ABC × XYZ (количество товаров):
Группа_XYZ     X    Y     Z
Группа_ABC                 
A           1301  210  2357
B           2186  225  3384
C           6679  236  5246

🎯 Количество товаров в сегменте CZ: 5246

📋 Примеры товаров из сегмента CZ:
['sku15863', 'sku2948', 'sku5612', 'sku28397', 'sku15273', 'sku10643', 'sku9449', 'sku2588', 'sku11133', 'sku24196'] ...


Какая средняя сумма покупки у одновизитника?
Перегруппировка данных из таблицы
Сгруппируйте данные по уникальности клиентов в первый столбец, а во второй — количество дней, когда посещал клиент (если в один день было несколько визитов, учитывайте только один раз). Затем посчитайте, сколько клиентов посетили один раз, суммируйте и объедините с общими расходами клиента за день, разделив на количество клиентов, посетивших в этот день, чтобы вычислить среднее значение.


In [9]:
import pandas as pd

# --- 1. Проверка: есть ли df?
if 'df' not in globals():
    raise NameError("❌ Переменная 'df' не найдена. Загрузите данные.")

# --- 2. Проверка необходимых колонок ---
required_cols = ['client', 'tr_date', 'amount']
for col in required_cols:
    if col not in df.columns:
        raise KeyError(f"❌ Нет колонки: '{col}'")

# --- 3. Преобразование даты ---
df['tr_date'] = pd.to_datetime(df['tr_date'], errors='coerce')
df = df.dropna(subset=['tr_date']).copy()

# --- 4. Создаём дату без времени ---
df['visit_date'] = df['tr_date'].dt.date  # тип: date

# --- 5. Шаг 1: Группировка по клиенту → сколько уникальных дней визитов ---
client_visit_days = (
    df.groupby('client')['visit_date']
    .nunique()
    .reset_index()
    .rename(columns={'visit_date': 'unique_visit_days'})
)

print(f"✅ Всего клиентов: {len(client_visit_days)}")

# --- 6. Шаг 2: Кто — одновизитник? (ровно 1 день) ---
one_visit_clients = client_visit_days[client_visit_days['unique_visit_days'] == 1]['client']
print(f"👥 Одновизитников (1 визит): {len(one_visit_clients)}")

# --- 7. Данные только по одновизитникам ---
df_one_visit = df[df['client'].isin(one_visit_clients)].copy()

# --- 8. Подсчёт суммы покупки для каждого визита (один день = одна покупка) ---
# Так как у одновизитника только один день — суммируем все его покупки за этот день
one_visit_amount = (
    df_one_visit.groupby(['client', 'visit_date'])['amount']
    .sum()
    .reset_index()
)

# --- 9. Средняя сумма покупки у одновизитника — это среднее по этим суммам
avg_check_one_visit = one_visit_amount['amount'].mean()

# --- 10. Дополнительно: средний чек в день (все клиенты) ---
# Сумма по дням
daily_total_amount = df.groupby('visit_date')['amount'].sum()
# Количество уникальных клиентов в день
daily_client_count = df.groupby('visit_date')['client'].nunique()

# Средний чек в день = оборот / число клиентов
daily_avg_check = (daily_total_amount / daily_client_count).rename('avg_check_per_client')

# Объединяем с данными одновизитников
one_visit_with_daily_avg = one_visit_amount.merge(
    daily_avg_check,
    left_on='visit_date',
    right_index=True
)

# --- 11. Вывод результата ---
print("\n" + "="*60)
print("🎯 ОТВЕТ НА ВОПРОС")
print("="*60)
print(f"Средняя сумма покупки у одновизитника: {avg_check_one_visit:,.2f}")

# --- 12. Дополнительная статистика ---
overall_daily_avg = one_visit_with_daily_avg['avg_check_per_client'].mean()
print(f"Средний чек в день (все клиенты):       {overall_daily_avg:,.2f}")

# Сравнение
if avg_check_one_visit > overall_daily_avg:
    status = "✅ Одновизитники тратят больше среднего"
else:
    status = "⚠️ Одновизитники тратят меньше среднего"

print(f"📊 {status}")




✅ Всего клиентов: 42746
👥 Одновизитников (1 визит): 12859

🎯 ОТВЕТ НА ВОПРОС
Средняя сумма покупки у одновизитника: 3,535.45
Средний чек в день (все клиенты):       2,454.44
📊 ✅ Одновизитники тратят больше среднего


Сколько в сети клиентов, которые совершили более 50 визитов за все время? Под визитом понимаем все чеки клиента в пределах одних суток.
Перегруппировка данных в таблице
Сгруппируйте по уникальности клиентов в первом столбце, во втором столбце — количество дней с уникальными посещениями клиентов (если посещений несколько в один день, учитывается только одно). Затем подсчитайте, сколько клиентов имеют количество посещений больше 50, суммируйте и определите, сколько всего клиентов удовлетворяют этому условию.

In [10]:
import pandas as pd

# --- 1. Проверка: есть ли df?
if 'df' not in globals():
    raise NameError("❌ Переменная 'df' не найдена. Загрузите данные.")

# --- 2. Проверка колонок ---
required_cols = ['client', 'tr_date', 'amount']
for col in required_cols:
    if col not in df.columns:
        raise KeyError(f"❌ Нет колонки: '{col}'")

# --- 3. Преобразование даты ---
df['tr_date'] = pd.to_datetime(df['tr_date'], errors='coerce')
df = df.dropna(subset=['tr_date']).copy()

# --- 4. Создаём дату без времени (визит = один день) ---
df['visit_date'] = df['tr_date'].dt.date

# --- 5. Перегруппировка: клиент → количество уникальных дней визитов ---
# Шаг 1: Группировка
client_unique_visits = (
    df.groupby('client')['visit_date']
    .nunique()  # считаем уникальные даты
    .reset_index()
    .rename(columns={'visit_date': 'unique_visit_days'})
)

print(f"✅ Обработано клиентов: {len(client_unique_visits)}")

# --- 6. Фильтр: более 50 визитов ---
clients_over_50 = client_unique_visits[client_unique_visits['unique_visit_days'] > 50]

count_over_50 = len(clients_over_50)

# --- 7. Вывод результата ---
print("\n" + "="*60)
print("🎯 КЛИЕНТЫ С БОЛЕЕ ЧЕМ 50 ВИЗИТОВ")
print("="*60)
print(f"📌 Количество клиентов, совершивших более 50 визитов: {count_over_50}")

if count_over_50 > 0:
    print(f"\n📋 Примеры таких клиентов (первые 10):")
    print(clients_over_50.sort_values(by='unique_visit_days', ascending=False).head(10))
else:
    print("💡 Нет клиентов с более чем 50 визитами.")

# --- 8. Дополнительно: статистика ---
print(f"\n📊 Общая статистика по количеству визитов:")
print(client_unique_visits['unique_visit_days'].describe().round(2))


✅ Обработано клиентов: 42746

🎯 КЛИЕНТЫ С БОЛЕЕ ЧЕМ 50 ВИЗИТОВ
📌 Количество клиентов, совершивших более 50 визитов: 275

📋 Примеры таких клиентов (первые 10):
            client  unique_visit_days
6283   client15654                174
36056  client42450                173
20306  client28276                169
30646  client37582                159
26757  client34081                157
6689   client16019                154
25653  client33088                144
31359  client38223                142
27536  client34783                133
38667   client6327                121

📊 Общая статистика по количеству визитов:
count    42746.00
mean         6.18
std          9.06
min          1.00
25%          1.00
50%          3.00
75%          7.00
max        174.00
Name: unique_visit_days, dtype: float64


Сколько товаров в продажах со средневзвешенной ценой 10 000 рублей и более?Составьте список всех товаров в первом столбце (без повторов), во втором столбце подсчитайте соответствующее количество, в третьем столбце подсчитайте общую сумму. Затем разделите общую сумму на количество, чтобы вычислить среднее значение для каждого товара, после чего проверьте, превышает ли оно 10000. Если превышает, увеличьте счетчик, в конце подсчитайте и выведите общее количество товаров, соответствующих условию.

In [11]:
import pandas as pd

# --- 1. Проверка: есть ли df?
if 'df' not in globals():
    raise NameError("❌ Переменная 'df' не найдена. Загрузите данные.")

# --- 2. Проверка колонок ---
required_cols = ['item', 'quantity', 'amount']
for col in required_cols:
    if col not in df.columns:
        raise KeyError(f"❌ Нет колонки: '{col}'")

# --- 3. Группировка: по товару — количество и сумма ---
product_summary = df.groupby('item').agg(
    quantity=('quantity', 'sum'),      # общее количество проданных единиц
    total_amount=('amount', 'sum')     # общая сумма
).reset_index()

# --- 4. Расчёт средневзвешенной цены ---
product_summary['avg_price'] = product_summary['total_amount'] / product_summary['quantity']

# Замена NaN (если quantity = 0)
product_summary['avg_price'] = product_summary['avg_price'].replace([float('inf'), -float('inf')], 0).fillna(0)

# --- 5. Фильтр: средняя цена >= 10 000
high_price_products = product_summary[product_summary['avg_price'] >= 10000]

# --- 6. Сортировка по убыванию цены (опционально)
high_price_products = high_price_products.sort_values(by='avg_price', ascending=False)

# --- 7. Вывод результата ---
count_high_price = len(high_price_products)

print("🎯 ТОВАРЫ СО СРЕДНЕЙ ЦЕНОЙ 10 000 РУБЛЕЙ И БОЛЕЕ")
print("="*80)
print(f"📌 Найдено товаров: {count_high_price}")
print("-" * 80)

if count_high_price > 0:
    print("📋 Список (первые 10):")
    print(
        high_price_products[['item', 'quantity', 'total_amount', 'avg_price']]
        .head(10)
        .round(2)
        .to_string(index=False)
    )
else:
    print("💡 Нет товаров со средней ценой 10 000 и более.")

# --- 8. Дополнительно: общая таблица (если нужна)
# print("\n📊 Полная таблица средних цен:")
# print(product_summary[['item', 'quantity', 'total_amount', 'avg_price']].round(2).head())


🎯 ТОВАРЫ СО СРЕДНЕЙ ЦЕНОЙ 10 000 РУБЛЕЙ И БОЛЕЕ
📌 Найдено товаров: 514
--------------------------------------------------------------------------------
📋 Список (первые 10):
    item  quantity  total_amount  avg_price
 sku8449         1        119500  119500.00
 sku6834         9        611419   67935.44
sku29862         3        192473   64157.67
sku27261         2        126980   63490.00
 sku5775         2        121981   60990.50
sku14468         2        119940   59970.00
sku24160         1         53991   53991.00
sku19447         7        361737   51676.71
sku16256         1         47491   47491.00
  sku906        12        558030   46502.50


Получите список товаров, у которых изменился сегмент ABC-XYZ (сравниваем два периода: все периоды и последние 6 мес.). Сравниваем только товары, которые продавались в обоих периодах!

Какой средний коэффициент вариации у этих товаров? Ответ округлите до одного знака после запятой.Сначала получите таблицу ABC-XYZ-анализа за весь период, затем добавьте условие для последних 6 месяцев, чтобы получить вторую таблицу ABC-XYZ-анализа. После этого объедините таблицы, используя столбец «клиент» из второй таблицы в качестве ключа соединения (так вы получите таблицу товаров, которые присутствуют в оба периода).

Далее нужно добавить условие: проверить, изменилась ли категория одного и того же товара (например, товар A за весь период имеет категорию AC и за последние 6 месяцев также имеет категорию AC — значит, изменений нет, и мы его удаляем).

Затем получаем таблицу товаров с изменившейся категорией. Для этой таблицы далее считаем сумму коэффициентов вариации по всем товарам и делим на общее количество товаров с изменениями, чтобы получить среднее значение коэффициента вариации.

In [4]:
import pandas as pd
import numpy as np

# --- 1. Проверка: есть ли df?
if 'df' not in globals():
    raise NameError("❌ Переменная 'df' не найдена. Загрузите данные.")

# --- 2. Проверка колонок ---
required_cols = ['item', 'tr_date', 'quantity', 'amount']
for col in required_cols:
    if col not in df.columns:
        raise KeyError(f"❌ Нет колонки: '{col}'")

# --- 3. Преобразование даты ---
df['tr_date'] = pd.to_datetime(df['tr_date'], errors='coerce')
df = df.dropna(subset=['tr_date']).copy()

# --- 4. Функция: ABC-XYZ анализ для заданного периода ---
def abc_xyz_analysis_period(data, period_name="Period"):
    # Группировка по item
    grouped = data.groupby('item')['amount'].sum().sort_values(ascending=False)
    total = grouped.sum()
    cumsum = grouped.cumsum()
    cum_percent = (cumsum / total) * 100

    # ABC: по сумме
    def assign_abc(cum):
        if cum <= 80:
            return 'A'
        elif cum <= 95:
            return 'B'
        else:
            return 'C'

    abc = pd.Series(cum_percent, name='cum_percent').apply(assign_abc)
    abc = pd.DataFrame({'ABC': abc})

    # XYZ: по стабильности quantity
    qty_stats = data.groupby('item')['quantity'].agg(['mean', 'std'])
    qty_stats = qty_stats[qty_stats['mean'] > 0]
    qty_stats['cv'] = (qty_stats['std'] / qty_stats['mean']) * 100
    qty_stats['cv'] = qty_stats['cv'].replace([np.inf, -np.inf], 0).fillna(0)

    def assign_xyz(cv):
        if cv < 10:
            return 'X'
        elif 10 <= cv < 25:
            return 'Y'
        else:
            return 'Z'

    xyz = pd.DataFrame({'XYZ': qty_stats['cv'].apply(assign_xyz)})
    xyz['cv'] = qty_stats['cv']  # сохраним CV для дальнейшего анализа

    # Объединяем
    result = abc.join(xyz, how='inner')
    result['ABC_XYZ'] = result['ABC'] + result['XYZ']
    result['period'] = period_name

    return result

# --- 5. Анализ за ВЕСЬ период ---
all_data = df.copy()
abc_xyz_all = abc_xyz_analysis_period(all_data, "all")

# --- 6. Анализ за ПОСЛЕДНИЕ 6 МЕСЯЦЕВ ---
last_date = df['tr_date'].max()
start_date_6m = last_date - pd.DateOffset(months=6)
recent_data = df[df['tr_date'] >= start_date_6m]
abc_xyz_6m = abc_xyz_analysis_period(recent_data, "recent")

# --- 7. Объединение: только товары, которые в обоих периодах ---
# Используем индекс (item) как ключ
common_items = abc_xyz_all[['ABC_XYZ']].join(
    abc_xyz_6m[['ABC_XYZ']], 
    how='inner', 
    lsuffix='_all', 
    rsuffix='_6m'
)

print(f"✅ Найдено товаров, продававшихся в обоих периодах: {len(common_items)}")

# --- 8. Фильтр: изменился ли сегмент?
changed = common_items[common_items['ABC_XYZ_all'] != common_items['ABC_XYZ_6m']].copy()
print(f"🔄 Товаров с изменившимся сегментом ABC-XYZ: {len(changed)}")

if len(changed) == 0:
    print("❌ Нет товаров с изменённым сегментом.")
else:
    # --- 9. Добавим коэффициент вариации из последнего периода ---
    # Берём CV из `abc_xyz_6m` (или можно из `all` — но логичнее из текущего)
    cv_from_6m = abc_xyz_6m['cv']
    changed_with_cv = changed.join(cv_from_6m, how='left')

    # --- 10. Средний коэффициент вариации ---
    avg_cv = changed_with_cv['cv'].mean()
    avg_cv_rounded = round(avg_cv, 1)

    print(f"\n🎯 Средний коэффициент вариации у товаров с изменением сегмента: {avg_cv_rounded}")


✅ Найдено товаров, продававшихся в обоих периодах: 21824
🔄 Товаров с изменившимся сегментом ABC-XYZ: 7512

🎯 Средний коэффициент вариации у товаров с изменением сегмента: 19.9
